<a href="https://colab.research.google.com/github/jhuarancca/ASU_DataMining/blob/main/DataMininProyecto03_ver02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np  

from scipy.stats import skew, kurtosis, entropy
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics.cluster import contingency_matrix

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import cluster, silhouette_score, v_measure_score, adjusted_rand_score, completeness_score

def calculatePurity(groundTruth, labels):
    # compute contingency matrix (also called confusion matrix)
    cont_mat = contingency_matrix(groundTruth, labels)
    # return purity
    return np.sum(np.amax(cont_mat, axis=0)) / np.sum(cont_mat)


def calculateSSE(X, labels,n):
    sse = 0
    for i in range(n):
        cluster = X[labels == i]
        mean = cluster.mean(axis=0)
        sse += ((cluster - mean) ** 2).sum()
    return sse


def calculateEntropy(groundTruth, labels, n):
    ent = 0
    for i in range(n):
        bincount = np.bincount(groundTruth[labels == i])
        w = (labels == i).sum()
        probabilities = bincount[bincount != 0] / w
        ent += entropy(probabilities) * w / labels.shape[0]
    return ent


bins = {1:[0,20],2:[21,40],3:[41,60],4:[61,80],5:[81,100],6:[101,120]}

def getBin(mealAmountGrams):
    lint_bin = 0
    li_binNumber =1
    for bin_number, bin_range in bins.items():
        if bin_range[0]<=mealAmountGrams<=bin_range[1]:
            li_binNumber = bin_number
    return li_binNumber

dfIns = pd.read_csv ('InsulinData.csv')
dfCGM = pd.read_csv ('CGMData.csv')


dfIns['Datetime']=pd.to_datetime(dfIns['Date'] + ' ' + dfIns['Time'])
dfCGM['Datetime']=pd.to_datetime(dfCGM['Date'] + ' ' + dfCGM['Time'])
dfIns.rename(columns={"BWZ Carb Input (grams)": "Carb"},inplace=True)

dfMeals=dfIns
dfMeals=dfMeals.drop(dfMeals[dfMeals['Carb'].isna()].index)
dfMeals = dfMeals.drop(dfMeals[(dfMeals['Carb']==0.0)].index)
dfMeals=dfMeals[["Datetime", "Carb"]]
dfMeals=dfMeals.reset_index(drop=True)
dfMeals = dfMeals.sort_values(by=['Datetime'], ascending=True)
dfMeals['Delete']='N'


dfMealsTmp=dfMeals
dfMealsTmp = dfMealsTmp.sort_values(by=['Datetime'], ascending=True)
dfMealsTmp=dfMealsTmp.reset_index(drop=True)

for index, row in dfMealsTmp.iterrows():
   ld_DateTime1=dfMealsTmp.at[index, 'Datetime']
   
   if index<(len(dfMealsTmp.index)-1):
    #print(index,len(dfMeals.index))
    ld_DateTime2=dfMealsTmp.at[index+1,'Datetime']
    ld_DateTime3=ld_DateTime1  + timedelta(hours=2)
    if ld_DateTime2<=ld_DateTime3:
      dfMeals.at[index,'Delete']='Y'
      #print(index,row['Datetime'], row['Carb'],ld_DateTime2,ld_DateTime3)
dfMeals01=dfMeals[(dfMeals['Delete']=='N')]
dfMeals01=dfMeals01.reset_index(drop=True)

import math

maxMeal = max(dfMeals01['Carb'])
minMeal = min(dfMeals01['Carb'])

print("Max meal value (grams):", maxMeal)
print("Min meal value (grams):", minMeal)

binsNumber = math.ceil((maxMeal-minMeal)/20)
print("In total you should have N = (",maxMeal,"-",minMeal,"/ 20) i.e.", binsNumber, "bins")

dfMeals01

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Max meal value (grams): 122.0
Min meal value (grams): 3.0
In total you should have N = ( 122.0 - 3.0 / 20) i.e. 6 bins


,Datetime,Carb,Delete
0,2017-07-25 10:21:19,91.0,N
1,2017-07-25 10:39:46,58.0,N
2,2017-07-25 18:31:40,115.0,N
3,2017-07-26 09:27:16,72.0,N
4,2017-07-26 19:15:06,60.0,N
...,...,...,...
599,2018-02-11 10:02:19,10.0,N
600,2018-02-11 12:43:23,27.0,N
601,2018-02-11 15:34:26,9.0,N
602,2018-02-11 20:33:18,71.0,N


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
dfMeals01["CarbScaled"] = scaler.fit_transform(dfMeals01['Carb'].values.reshape(-1,1))

dfMeals01['GroundTruth'] = dfMeals01['Carb'].apply(lambda x: getBin(x)) 
#dfMeals01["GroundTruth"].value_counts().sort_values(ascending=True).plot(kind="barh")

X = dfMeals01["CarbScaled"].values.reshape(-1,1)

kmeans = KMeans(n_clusters=5, random_state=42, max_iter=100)
kmeans_model = kmeans.fit(X)
kmeans_silhouette = silhouette_score(X, kmeans.labels_).round(2)

# Create a column of K-means Prediction
dfMeals01['KmeanCluster'] = kmeans_model.predict(X)
# Calculate accuracy using entropy, purity_score, & v-measure score
kmeanEntropy = calculateEntropy(dfMeals01['GroundTruth'],dfMeals01['KmeanCluster'],binsNumber)
kmeanPurity = calculatePurity(dfMeals01['GroundTruth'],dfMeals01['KmeanCluster'])
kmeanSSE = calculateSSE(X, kmeans.labels_,binsNumber)

# Define a model
dbscan = DBSCAN(eps=0.3)
dbscan_model = dbscan.fit(X)
dbscan_silhouette = silhouette_score(X, dbscan_model.labels_).round(2)
dfMeals01['DBSCAN_Cluster'] = dbscan_model.fit_predict(X)

# Calculate accuracy using entropy, purity_score, & v-measure score
dbscanEntropy = calculateEntropy(dfMeals01['GroundTruth'],dfMeals01['DBSCAN_Cluster'],binsNumber)
dbscanPurity = calculatePurity(dfMeals01['GroundTruth'],dfMeals01['DBSCAN_Cluster'])
dbscanSSE = calculateSSE(X, dbscan.labels_,binsNumber)

dsResult = {}

dsResult['SSE for Kmeans'] =  "%.2f"%kmeanSSE
dsResult['SSE for DBSCAN'] =  "%.2f"%dbscanSSE
dsResult['Entropy for Kmeans'] =  "%.2f"%kmeanEntropy
dsResult['Entropy for DBSCAN'] =  "%.2f"%dbscanEntropy
dsResult['Purity for K means'] =  "%.2f"%kmeanPurity
dsResult['Purity for DBSCAN'] =  "%.2f"%dbscanPurity

dsResult = pd.DataFrame(dsResult, index=[0])
dsResult.to_csv('Result.csv',index=False)
dsResult

array([[ 2.14976453],
       [ 0.80699537],
       [ 3.12632391],
       [ 1.37665501],
       [ 0.88837532],
       [-0.57646376],
       [ 0.35940565],
       [ 0.64423547],
       [ 1.41734499],
       [ 1.29527506],
       [ 3.00425399],
       [-0.57646376],
       [-0.45439384],
       [ 1.70217481],
       [ 0.52216555],
       [-0.94267353],
       [ 3.41115373],
       [-0.77991363],
       [-0.57646376],
       [ 0.2780257 ],
       [ 2.10907455],
       [ 0.07457583],
       [-1.10543343],
       [ 0.07457583],
       [-0.57646376],
       [ 1.37665501],
       [ 1.70217481],
       [ 1.74286478],
       [-0.04749409],
       [ 0.07457583],
       [ 0.48147557],
       [ 0.40009563],
       [-0.53577379],
       [-0.00680412],
       [ 1.70217481],
       [-1.1461234 ],
       [ 0.92906529],
       [-0.33232391],
       [ 0.48147557],
       [-0.41370386],
       [-1.22750335],
       [ 1.25458509],
       [ 0.11526581],
       [ 0.35940565],
       [-0.33232391],
       [ 0